In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
films = pd.read_csv('rotten_tomatoes_movies.csv')

In [3]:
films.head(40)

,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix
0,space-zombie-bingo,Space Zombie Bingo!,50.0,NaN,NaN,NaN,NaN,2018-08-25,75.0,"Comedy, Horror, Sci-fi",English,George Ormrod,"George Ormrod,John Sabotta",NaN,NaN,NaN
1,the_green_grass,The Green Grass,NaN,NaN,NaN,NaN,NaN,2020-02-11,114.0,Drama,English,Tiffany Edwards,Tiffany Edwards,NaN,NaN,NaN
2,love_lies,"Love, Lies",43.0,NaN,NaN,NaN,NaN,NaN,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",NaN,NaN,NaN
3,the_sore_losers_1997,Sore Losers,60.0,NaN,NaN,NaN,NaN,2020-10-23,90.0,"Action, Mystery & thriller",English,John Michael McCarthy,John Michael McCarthy,NaN,NaN,NaN
4,dinosaur_island_2002,Dinosaur Island,70.0,NaN,NaN,NaN,NaN,2017-03-27,80.0,"Fantasy, Adventure, Animation",English,Will Meugniot,John Loy,NaN,NaN,NaN
5,adrift_2018,Adrift,65.0,69.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",$31.4M,STX Films,NaN
6,malta-con-huevo-scrambled-beer,Scrambled Beer,55.0,NaN,NaN,NaN,NaN,2015-10-05,88.0,Comedy,Spanish,Cristobal Valderrama,Cristobal Valderrama,NaN,NaN,NaN
7,kakabakaba-ka-ba,Kakabakaba ka ba? (Will Your Heart Beat Faster?),88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mike de Leon,NaN,NaN,NaN,NaN
8,sundowning,Sundowning,NaN,NaN,NaN,NaN,NaN,NaN,123.0,Drama,English,Jim Comas Cole,NaN,NaN,NaN,NaN
9,1035316-born_to_kill,Born to Kill,74.0,83.0,NaN,NaN,1947-04-30,2016-05-23,92.0,"Crime, Drama",English,Robert Wise,"Eve Greene,Richard Macaulay",NaN,NaN,NaN


In [4]:
films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143258 entries, 0 to 143257
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    143258 non-null  object 
 1   title                 142891 non-null  object 
 2   audienceScore         73248 non-null   float64
 3   tomatoMeter           33877 non-null   float64
 4   rating                13991 non-null   object 
 5   ratingContents        13991 non-null   object 
 6   releaseDateTheaters   30773 non-null   object 
 7   releaseDateStreaming  79420 non-null   object 
 8   runtimeMinutes        129431 non-null  float64
 9   genre                 132175 non-null  object 
 10  originalLanguage      129400 non-null  object 
 11  director              139064 non-null  object 
 12  writer                90116 non-null   object 
 13  boxOffice             14743 non-null   object 
 14  distributor           23005 non-null   object 
 15  

In [5]:
films.shape

(143258, 16)

In [6]:
films.title.isna().sum()

367

I will drop rows without titles

In [7]:
films = films.dropna(subset=['title'])

In [8]:
films.title.isna().sum()

0

In [9]:
films.isna().sum() / len(films) * 100

id                       0.000000
title                    0.000000
audienceScore           48.738549
tomatoMeter             76.291719
rating                  90.212120
ratingContents          90.212120
releaseDateTheaters     78.471702
releaseDateStreaming    44.447166
runtimeMinutes           9.463157
genre                    7.546312
originalLanguage         9.488351
director                 2.730053
writer                  36.963140
boxOffice               89.683745
distributor             83.905214
soundMix                88.863539
dtype: float64

I will encode both NaN for audiencaScore and TomatoMeter as 0, so that I can recognize the rows where there is no rating in the future. I will exclude these in the analysis of stats for ratings

In [10]:
films['audienceScore'].fillna(0, inplace=True)
films['tomatoMeter'].fillna(0, inplace=True)

In [11]:
films['rating'].value_counts()

R        7733
PG-13    3445
PG       1910
TVPG      424
TV14      395
TVMA       57
NC-17      19
TVG         1
TVY7        1
G           1
Name: rating, dtype: int64

I am going to group all less frequent ratings in other and replace the NaNs with the string 'unknown'

In [12]:
films['rating'].fillna('unknown', inplace=True)

In [13]:
def filter_ratings(x):
    if x in ['R', 'PG-13', 'PG', 'unknown']:
        return x
    else:
        return 'other'

In [14]:
films['rating'] = films['rating'].apply(filter_ratings)

In [15]:
films['rating'].value_counts(dropna=False)

unknown    128905
R            7733
PG-13        3445
PG           1910
other         898
Name: rating, dtype: int64

In [16]:
films['ratingContents'].value_counts()

['Language']                                                                                   365
['V']                                                                                          154
['Some Language']                                                                              143
['Some Violence']                                                                              126
['Violence']                                                                                   105
                                                                                              ... 
['Language', 'Drug Material', 'Nudity', 'Sexual Content', 'Strong Violence']                     1
['A Disturbing Image', 'Some Thematic Material']                                                 1
['Pervasive Sexual Content', 'Aberrant Sexuality', 'Some Drug Material', 'Strong Language']      1
['Drinking', 'Crude Humor', 'Language', 'Strong Sexual Content']                                 1
['Language

Going to replace the NaN in films['ratingContents'] with empty list to maintain the same data type

In [17]:
films['ratingContents'] = films['ratingContents'].apply(lambda x: [] if pd.isna(x) else x)

In [18]:
films['releaseDateTheaters'].dtype

dtype('O')

In [19]:
films['releaseDateTheaters'] = pd.to_datetime(films['releaseDateTheaters'], errors='coerce')

In [20]:
films['releaseDateTheaters'].dtype

dtype('<M8[ns]')

In [21]:
films['releaseDateTheaters'].isna().sum()

112129

I am going to replace the NaNs with 1800-01-01, so I will able to recognize them in the future

In [22]:
films['releaseDateTheaters'].fillna(pd.to_datetime('1800-01-01'), inplace=True)

Same for releaseDateStreaming

In [23]:
films['releaseDateStreaming'] = pd.to_datetime(films['releaseDateStreaming'], errors='coerce')
films['releaseDateStreaming'].fillna(pd.to_datetime('1800-01-01'), inplace=True)

In [24]:
films.isna().sum() / len(films) * 100

id                       0.000000
title                    0.000000
audienceScore            0.000000
tomatoMeter              0.000000
rating                   0.000000
ratingContents           0.000000
releaseDateTheaters      0.000000
releaseDateStreaming     0.000000
runtimeMinutes           9.463157
genre                    7.546312
originalLanguage         9.488351
director                 2.730053
writer                  36.963140
boxOffice               89.683745
distributor             83.905214
soundMix                88.863539
dtype: float64

In [25]:
films['runtimeMinutes'].describe()

count    129369.000000
mean         93.708841
std          28.128291
min           1.000000
25%          84.000000
50%          92.000000
75%         103.000000
max        2700.000000
Name: runtimeMinutes, dtype: float64

I will replace NaNs with 0, so I can still recognize them

In [26]:
films['runtimeMinutes'].fillna(0, inplace=True)

In [27]:
films['genre'].value_counts()

Drama                                          27842
Documentary                                    15155
Comedy                                         11509
Mystery & thriller                              7009
Comedy, Drama                                   5476
                                               ...  
Fantasy, Drama, Musical                            1
Holiday, Drama, Musical                            1
Drama, War, Adventure, Action                      1
Action, Adventure, Comedy, Drama, Animation        1
Western, Comedy, Animation                         1
Name: genre, Length: 2911, dtype: int64

In [28]:
films['genre'].fillna('unknown', inplace=True)

In [29]:
films['originalLanguage'].value_counts()

English            84989
Spanish             4785
Japanese            3480
Hindi               3307
French (Canada)     3281
                   ...  
Bhojpuri               1
Hawaiian               1
Yoruba                 1
smi                    1
Aramaic                1
Name: originalLanguage, Length: 112, dtype: int64

In [30]:
films['originalLanguage'].fillna('unknown', inplace=True)

In [31]:
films['originalLanguage'].value_counts()

English     84989
unknown     13558
Spanish      4785
Japanese     3480
Hindi        3307
            ...  
Bhojpuri        1
Yoruba          1
Hawaiian        1
smi             1
Aramaic         1
Name: originalLanguage, Length: 113, dtype: int64

In [32]:
films['director'].value_counts()

Unknown Director                                  3541
David DeCoteau                                     129
Sam Newfield                                       123
Fred Olen Ray                                       93
Joseph Kane                                         84
                                                  ... 
Kevin Higgins                                        1
Laurence Jarvik                                      1
Anna-Elizabeth Shakespeare,Hillary Shakespeare       1
Amy French                                           1
Peter Georgi                                         1
Name: director, Length: 62187, dtype: int64

In [33]:
films['director'].fillna('Unknown Director', inplace=True)

In [34]:
films['writer'].value_counts()

Jing Wong                        48
Barbara Kymlicka                 46
Woody Allen                      38
Kuang Ni                         37
Christine Conradt                37
                                 ..
Kuba Mikurda                      1
Gail Parent                       1
Brian Miller                      1
Jo Enaje                          1
David J. Francis,Mike Masters     1
Name: writer, Length: 67249, dtype: int64

In [35]:
films['writer'].fillna('Unknown Writer', inplace=True)

In [36]:
films['boxOffice'].value_counts()

$1.1M      118
$1.2M      100
$1.0M       97
$1.3M       97
$1.6M       70
          ... 
$228.4M      1
$86.5K       1
$160.4K      1
$129.4K      1
$218.3K      1
Name: boxOffice, Length: 4862, dtype: int64

Going to replace NaNs with 0, so I will remember in the future

In [37]:
films['boxOffice'].fillna('0', inplace=True)

In [38]:
def convert_dollars_to_floats(x):
    no_dollar = x.replace('$', '')
    if 'M' in no_dollar:
        float_n = float(no_dollar.replace('M', '')) * 1000000
    elif 'K' in no_dollar:
        float_n = float(no_dollar.replace('K', '')) * 1000
    else:
        float_n = float(no_dollar)   
    int_n = int(float_n)
    return int_n

In [39]:
convert_dollars_to_floats('$86.5K')

86500

In [40]:
films['boxOffice'] = films['boxOffice'].apply(convert_dollars_to_floats)

In [41]:
films['distributor'].value_counts()

Paramount Pictures                                                                                                           994
20th Century Fox                                                                                                             745
Universal Pictures                                                                                                           737
Warner Bros. Pictures                                                                                                        668
Metro-Goldwyn-Mayer                                                                                                          603
                                                                                                                            ... 
Tartan Films USA, TLA Releasing                                                                                                1
Cinerama Releasing Corporation [us], Anchor Bay Entertainment                                    

In [42]:
films['soundMix'].value_counts()

Surround                                                    4074
Dolby Digital                                               2374
Stereo                                                      2080
Mono                                                        1246
Stereo, Surround                                             473
                                                            ... 
Dolby, Surround, Dolby A                                       1
Magnetic Stereo 6 Track, Dolby A, Surround, Dolby Stereo       1
Dolby Stereo, Dolby SRD, SDDS                                  1
Vitaphone                                                      1
Dolby SR, Dolby SRD                                            1
Name: soundMix, Length: 551, dtype: int64

I am going to drop both soundMix and distributor because they contain many NaNs and do not provide essential information for this research

In [43]:
films.drop(['distributor', 'soundMix'], axis=1, inplace=True)

In [44]:
films.isna().sum() / len(films) * 100

id                      0.0
title                   0.0
audienceScore           0.0
tomatoMeter             0.0
rating                  0.0
ratingContents          0.0
releaseDateTheaters     0.0
releaseDateStreaming    0.0
runtimeMinutes          0.0
genre                   0.0
originalLanguage        0.0
director                0.0
writer                  0.0
boxOffice               0.0
dtype: float64

In [45]:
films.to_csv('films_NaNs_treated.csv', index=False)